In [1]:
import requests
import json
import pandas as pd

In [2]:
url = "https://www.binance.com/exchange-api/v2/public/asset-service/product/get-products"
data = requests.get(url).json()

In [3]:
def human_format(num):
    magnitude = 0
    while abs(num) >= 1000:
        magnitude += 1
        num /= 1000.0
    return '%.2f%s' % (num, ['', 'k', 'M', 'B', 'T'][magnitude])

def get_crypto_data(data, i):
    symbol = data['data'][i]['s']
    coin = data['data'][i]['b']
    base_coin = data['data'][i]['q']
    try:
        price = float(data['data'][i]['c'])
    except:
        price = None
    try:
        volume = int(data['data'][i]['cs'])
    except:
        volume = None
    
    #market_cap = price*volume

    return symbol, coin, base_coin, price, volume

In [4]:
index_list = []
for i in range(len(data['data'])):
    index_list.append(get_crypto_data(data, i))

In [5]:
cols = ['symbol', 'coin', 'base_coin', 'price', 'volume']
df_crypto = pd.DataFrame(index_list, columns = cols)

In [6]:
# Get only binance Stablecoins pairs
df_crypto = df_crypto[df_crypto['base_coin'].isin(['BUSD'])].reset_index(drop = True)

In [7]:
# Fiat crypto volume is equal to 0
df_crypto['volume'] = df_crypto['volume'].fillna(0)

In [8]:
# Calculate market capitalization for every coin
df_crypto['marketcap'] = df_crypto['price'] * df_crypto['volume']

In [9]:
df_crypto = df_crypto.sort_values(by = 'marketcap', ascending = False).reset_index(drop = True)

In [10]:
df_crypto_final = df_crypto.head(10).drop(columns = ['base_coin', 'symbol']).set_index('coin')

In [11]:
# Change pandas default format to one that is easier to read
pd.set_option('display.float_format', lambda x: f'{human_format(x)}')

In [12]:
body_message = f"Hello!! Here's an update of the top 10 cryptocurrencies on Binance:\n\n\
{str(df_crypto_final)}"

In [14]:
df_crypto_final

,price,volume,marketcap
coin,,,
BTC,22.38k,19.31M,432.04B
ETH,1.56k,122.37M,191.04B
BNB,286.30,157.89M,45.21B
XRP,0.36,50.95B,18.43B
ADA,0.33,34.68B,11.57B
DOGE,0.07,132.67B,9.79B
MATIC,1.12,8.73B,9.78B
SOL,20.82,382.28M,7.96B
DOT,5.90,1.16B,6.87B


In [15]:
from twilio.rest import Client 
 
account_sid = 'AC93ba471496881f9cc0db3202baec8afc' 
auth_token = 'zzZzzzZZzzzzZzZZZ' 
client = Client(account_sid, auth_token) 
 
message = client.messages.create( 
                              from_='whatsapp:+14155238886',  
                              body= body_message,      
                              to='whatsapp:+573502695238' 
                          ) 
 
print(message.sid)

SMa5ab22925ab732a0857e38b9698e4337
